In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from prize_grid_env import PrizeGridEnv
from gymnasium.envs.registration import register
import gymnasium as gym
from dqn import DQN
import matplotlib
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
register(
    id='PrizeGridEnv',  # Unique identifier for your environment
    entry_point='prize_grid_env:PrizeGridEnv',  # Module path and class name
)

In [15]:
env = gym.make('PrizeGridEnv')
type(env.action_space.sample())

numpy.int64

In [22]:
dqn = DQN([906, 1000, 1000, 1000, 4], 'PrizeGridEnv', grid_size=(30, 30), num_prizes=4)

In [23]:
import numpy as np
import torch
import wandb

wandb.init(project="submodular_nn")

# Define parameters for training
num_episodes = 1000          # Number of episodes to train
max_steps = 200              # Maximum steps per episode
batch_size = 32              # Batch size for training
target_update_frequency = 10 # Frequency to update the target network

# Training loop
for episode in range(num_episodes):
    state, prizes = dqn.env.reset()   # Reset the environment and get the initial state
    if episode == 0:
        print(prizes)
    # print(dqn.env.render())

    total_reward = 0           # Initialize total reward for this episode

    total_loss = 0
    for step in range(max_steps):
        action = dqn.select_action(state)  # Select an action using the policy
        next_state, reward, done, _ = dqn.env.step(action)  # Take a step in the environment
        
        # Store experience in replay memory
        dqn.replay_memory.add(state, action, reward, next_state, done)

        # Train the network if enough experiences are collected
        if len(dqn.replay_memory) >= batch_size:
            total_loss += dqn.train_step(batch_size)  # Perform a training step

        total_reward += reward  # Accumulate reward
        state = next_state      # Transition to the next state

        if done:
            break  # End the episode if done

    # Update target network at specified intervals
    if episode % target_update_frequency == 0:
        dqn.update_target(soft=True)

    # clamp weights 
    dqn.online_net.clamp_weights()
    # dqn.target_net.clamp_weights()

    dqn.eps -= dqn.eps_decay_amt
    print(dqn.eps)

    print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}, Total Loss: {total_loss}")
    wandb.log({
        "episode": episode + 1,
        "Reward": total_reward, 
        "Loss": total_loss,
    })

# Close the environment after training
dqn.env.close()

{'max reward': 330}
0.198
Episode 1/1000, Total Reward: 25, Total Loss: 8.537961840629578
0.196
Episode 2/1000, Total Reward: 65, Total Loss: 17.203999564051628
0.194
Episode 3/1000, Total Reward: 61, Total Loss: 18.417672485113144
0.192
Episode 4/1000, Total Reward: 73, Total Loss: 16.217585988342762
0.19
Episode 5/1000, Total Reward: 66, Total Loss: 14.869926668703556
0.188
Episode 6/1000, Total Reward: 73, Total Loss: 13.794516049325466
0.186
Episode 7/1000, Total Reward: 89, Total Loss: 13.210798658430576
0.184
Episode 8/1000, Total Reward: 62, Total Loss: 12.291025303304195
0.182
Episode 9/1000, Total Reward: 44, Total Loss: 11.972819730639458
0.18
Episode 10/1000, Total Reward: 73, Total Loss: 12.231687508523464
0.178
Episode 11/1000, Total Reward: 37, Total Loss: 12.101465038955212
0.176
Episode 12/1000, Total Reward: 34, Total Loss: 12.18221016973257
0.174
Episode 13/1000, Total Reward: 62, Total Loss: 12.081464529037476
0.172
Episode 14/1000, Total Reward: 69, Total Loss: 12.3